# EN to MS longer text

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/en-ms-long-translation](https://github.com/huseinzol05/Malaya/tree/master/example/en-ms-long-translation).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
%%time
import malaya

CPU times: user 5.86 s, sys: 986 ms, total: 6.85 s
Wall time: 6.26 s


### List available Transformer models

In [2]:
malaya.translation.en_ms.available_transformer()

,Size (MB),Quantized Size (MB),BLEU,SacreBLEU Verbose,SacreBLEU-chrF++-FLORES200,Suggested length
small,42.7,13.4,58.67129,80.2/63.8/52.8/44.4 (BP = 0.997 ratio = 0.997 ...,64.46,256
base,234,82.7,68.259569,86.3/73.3/64.1/56.8 (BP = 0.985 ratio = 0.985 ...,66.28,256
bigbird,246,63.7,59.863535,82.2/65.9/54.9/46.4 (BP = 0.982 ratio = 0.982 ...,59.64,1024
small-bigbird,50.4,13.1,56.701338,80.7/63.2/51.6/42.8 (BP = 0.979 ratio = 0.979 ...,58.01,1024
noisy-base,234,82.7,67.285716,86.1/72.7/63.3/55.8 (BP = 0.981 ratio = 0.981 ...,66.2,256


If you look at `Suggested length`, bigbird 4x longer than normal transformer models, able to infer longer text without need to partition. Let's we do some examples.

We are going to compare `small` and `base` models with `small-bigbird` and `bigbird` models for MS-EN translation task. Feel free to test on EN-MS translation.

### Load Transformer models

```python
def transformer(model: str = 'base', quantized: bool = False, **kwargs):
    """
    Load Transformer encoder-decoder model to translate EN-to-MS.

    Parameters
    ----------
    model : str, optional (default='base')
        Model architecture supported. Allowed values:

        * ``'small'`` - Transformer SMALL parameters.
        * ``'base'`` - Transformer BASE parameters.
        * ``'bigbird'`` - BigBird BASE parameters.
        * ``'small-bigbird'`` - BigBird SMALL parameters.
        * ``'noisy-base'`` - Transformer BASE parameters trained on noisy dataset.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: model
        List of model classes:

        * if `bigbird` in model, return `malaya.model.bigbird.Translation`.
        * else, return `malaya.model.tf.Translation`.
    """
```

In [3]:
transformer = malaya.translation.en_ms.transformer()
transformer_small = malaya.translation.en_ms.transformer(model = 'small')

In [8]:
bigbird = malaya.translation.en_ms.transformer(model = 'bigbird')
quantized_bigbird = malaya.translation.en_ms.transformer(model = 'bigbird', quantized = True)

In [9]:
bigbird_small = malaya.translation.en_ms.transformer(model = 'small-bigbird')
quantized_bigbird_small = malaya.translation.en_ms.transformer(model = 'small-bigbird', quantized = True)

### Long text examples

In [13]:
string = """
(Reuters) - The Nicaraguan government ordered the closure of a radio station known for its opposition against President Daniel Ortega, its director said on Friday.

Local radio station Radio Dario was shut on Friday, its director Anibal Toruno said on his Twitter account, showing images of the government's decision.

Radio Dario was considered one of the last operating news radio stations critical of Ortega, whose government recently shut down seven Catholic radio stations linked to a bishop critical of Ortega.

"Neither turning off equipment nor arbitrarily withdrawing a license will succeed in silencing us or silencing the truth. Far from intimidating us, their actions strengthen us," Toruno said.

The Nicaraguan government could not be reached immediately for comment.

The Nicaraguan telecommunications agency, in the statement posted by Toruno, said the radio station did not meet the technical requirements to be on air.

(Reporting by Ismael Lopez; Writing by Carolina Pulice; Editing by William Mallard)
"""

In [7]:
string2 = """
KUALA LUMPUR (Aug 13): The government will focus on putting the littoral combat ship (LCS) project back on track as decided by the Cabinet, said Senior Defence Minister Datuk Seri Hishammuddin Hussein in a post on Facebook.

He said negotiations with vendors, original equipment manufacturers (OEM) and banks are being carried out.

Hishammuddin said this in relation to the procurement of the LCS which became controversial following the Public Accounts Committee (PAC) report revealing that not a single ship had been completed even though the project's contract, worth RM6.083 billion, stipulated that five of the six ships should be completed and handed over by August 2022.

In the posting, Hishammuddin said his team has taken media personnel to Boustead Naval Shipyard in Lumut, Perak on Saturday (Aug 13) to take a look at the external and internal conditions of the LCS which is under construction.

"I never get tired of asserting many times that the Defence Ministry and I have nothing to hide. At the same time, the wrongdoers must be brought to justice," he added.
"""

do simple cleaning, remove newline and weird characters.

In [10]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [14]:
string = cleaning(string)
string2 = cleaning(string2)

In [15]:
len(string.split()), len(string2.split())

(154, 176)

### Translate using greedy decoder

In [17]:
from pprint import pprint

In [18]:
pprint(transformer.greedy_decoder([string]))

['(Reuters) - Pemerintah Nikaragua memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, kata pengarahnya Anibal Toruno di akaun Twitternya, menunjukkan '
 'gambar keputusan pemerintah. Radio Dario dianggap sebagai salah satu stesen '
 'radio berita operasi terakhir yang mengkritik Ortega, yang pemerintahnya '
 'baru-baru ini menutup tujuh stesen radio Katolik yang dikaitkan dengan uskup '
 'yang mengkritik Ortega. "Tidak mematikan peralatan atau secara '
 'sewenang-wenangnya menarik lesen akan berjaya membungkam kita atau '
 'membungkam kebenaran. Jauh dari menakut-nakutkan kita, tindakan mereka '
 'menguatkan kita," kata Toruno. Kerajaan Nikaragua tidak dapat dihubungi '
 'segera untuk mendapatkan komen. Agensi telekomunikasi Nikaragua, dalam '
 'kenyataan yang disiarkan oleh Toruno, mengatakan stesen radio itu tidak '
 'memenu

In [19]:
pprint(transformer_small.greedy_decoder([string]))

['(Reuters) - Pemerintah Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, pengarahnya Anibal Toruno mengatakan di akaun Twitternya, '
 'menunjukkan gambar keputusan pemerintah. Radio Dario dianggap salah satu '
 'stesen radio operasi terakhir yang mengkritik Ortega, yang pemerintahnya '
 'baru-baru ini menutup tujuh stesen radio Katolik yang dikaitkan dengan uskup '
 'kritikal Ortega. "Tidak ada yang mematikan peralatan atau sewenang-wenangnya '
 'menarik lesen akan berjaya membungkam kami atau membungkam kebenaran. Jauh '
 'dari menakutkan kami, tindakan mereka menguatkan kami," kata Toruno. '
 'Kerajaan Nicaraguan tidak dapat dihubungi segera untuk memberi komen. Agensi '
 'telekomunikasi Nicaraguan, dalam kenyataan yang disiarkan oleh Toruno, '
 'mengatakan stesen radio tidak memenuhi syarat teknikal untuk disi

**Normal transformer models not able to infer long texts, we need to partition the text, example, split the text by full stop**.

In [20]:
partition_string = malaya.text.function.split_into_sentences(string)
partition_string

['(Reuters) - The Nicaraguan government ordered the closure of a radio station known for its opposition against President Daniel Ortega, its director said on Friday.',
 "Local radio station Radio Dario was shut on Friday, its director Anibal Toruno said on his Twitter account, showing images of the government's decision.",
 'Radio Dario was considered one of the last operating news radio stations critical of Ortega, whose government recently shut down seven Catholic radio stations linked to a bishop critical of Ortega.',
 '"Neither turning off equipment nor arbitrarily withdrawing a license will succeed in silencing us or silencing the truth.',
 'Far from intimidating us, their actions strengthen us," Toruno said.',
 'The Nicaraguan government could not be reached immediately for comment.',
 'The Nicaraguan telecommunications agency, in the statement posted by Toruno, said the radio station did not meet the technical requirements to be on air.',
 '(Reporting by Ismael Lopez; Writing by

In [21]:
pprint(transformer.greedy_decoder(partition_string))

['(Reuters) - Pemerintah Nikaragua memerintahkan penutupan stasiun radio yang '
 'dikenal karena oposisi terhadap Presiden Daniel Ortega, kata pengarahnya '
 'pada Jumat.',
 'Stesen radio tempatan Radio Dario ditutup pada Jumaat, kata pengarahnya '
 'Anibal Toruno di akaun Twitternya, menunjukkan imej keputusan kerajaan itu.',
 'Radio Dario dianggap sebagai salah satu stesen radio berita operasi terakhir '
 'yang kritikal terhadap Ortega, yang kerajaannya baru-baru ini menutup tujuh '
 'stesen radio Katolik yang dikaitkan dengan biskop yang kritikal terhadap '
 'Ortega.',
 '"Tidak mematikan peralatan mahupun dengan sewenang-wenangnya menarik balik '
 'lesen akan berjaya membungkam kami atau membungkam kebenaran.',
 'Jauh dari menakut-nakutkan kami, tindakan mereka memperkuat kami," kata '
 'Toruno.',
 'Pemerintah Nikaragua tidak dapat dihubungi segera untuk mendapatkan '
 'komentar.',
 'Agensi telekomunikasi Nicaragua, dalam kenyataan yang dimuat naik Toruno, '
 'berkata stesen radio i

In [22]:
pprint(transformer_small.greedy_decoder(partition_string))

['(Reuters) - Pemerintah Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat.',
 'Stesen radio tempatan Radio Dario ditutup pada hari Jumaat, pengarahnya '
 'Anibal Toruno berkata di akaun Twitternya, menunjukkan gambar keputusan '
 'kerajaan.',
 'Radio Dario dianggap salah satu stesen radio berita operasi terakhir yang '
 'mengkritik Ortega, yang baru-baru ini pemerintahnya menutup tujuh stesen '
 'radio Katolik yang dikaitkan dengan seorang uskup yang kritikal terhadap '
 'Ortega.',
 '"Tidak ada yang mematikan peralatan atau sewenang-wenangnya menarik lesen '
 'akan berjaya membungkam kami atau membungkam kebenaran.',
 'Jauh dari mengintimidasi kami, tindakan mereka memperkuat kami," kata '
 'Toruno.',
 'Kerajaan Nicaraguan tidak dapat dihubungi segera untuk memberi komen.hari '
 'ini.anda tidak',
 'Agensi telekomunikasi Nicaraguan, dalam kenyataan yang disiarkan oleh '
 'Toruno

### Problem with partitioning

Problem with partition, the model assumed N + 1 element has no relationship with N element, vice versa. Attention mechanism will not work across partition.

So we introduced BigBird to solve this problem.

In [23]:
pprint(bigbird_small.greedy_decoder([string]))

['(Reuters) - Pemerintah Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, pengarahnya Anibal Toruno mengatakan di akaun Twitternya, '
 'menunjukkan gambar keputusan pemerintah. Radio Dario dianggap sebagai salah '
 'satu stesen radio berita operasi terakhir yang kritikal terhadap Ortega, '
 'yang pemerintahnya baru-baru ini menutup tujuh stesen radio Katolik yang '
 'dikaitkan dengan biskop yang kritikal terhadap Ortega. "Sama ada mematikan '
 'peralatan dan sewenang-wenangnya menarik lesen akan berjaya merampas kami '
 'atau melepaskan hukuman kebenaran. Jauh dari menakutkan kami, tindakan '
 'mereka memperkuat kami," kata Toruno. Pemerintah Nicaraguan tidak dapat '
 'dihubungi segera untuk memberi komen. Agensi telekomunikasi Nicaraguan, '
 'dalam kenyataan yang disiarkan oleh Toruno, mengatakan stesen radio tid

In [24]:
pprint(quantized_bigbird_small.greedy_decoder([string]))

['(Reuters) - Pemerintah Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, pengarahnya Anibal Toruno mengatakan di akaun Twitternya, '
 'menunjukkan gambar keputusan pemerintah. Radio Dario dianggap sebagai salah '
 'satu stesen radio berita operasi terakhir yang kritikal terhadap Ortega, '
 'yang pemerintahnya baru-baru ini menutup tujuh stesen radio Katolik yang '
 'dikaitkan dengan uskup kritikal terhadap Ortega. "Tidak dapat mematikan '
 'peralatan dan sewenang-wenangnya menarik lesen akan berjaya merampas kami '
 'atau merampas kebenaran. Jauh dari menakutkan kami, tindakan mereka '
 'memperkuat kami," kata Toruno. Pemerintah Nicaraguan tidak dapat dihubungi '
 'segera untuk memberi komen. Agensi telekomunikasi Nicaraguan, dalam '
 'kenyataan yang disiarkan oleh Toruno, mengatakan stesen radio tidak memenuhi '

In [25]:
pprint(bigbird.greedy_decoder([string]))

['(Reuters) - Kerajaan Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, kata pengarahnya Anibal Toruno di akaun Twitternya, menunjukkan '
 'gambar keputusan pemerintah. Radio Dario dianggap sebagai salah satu stesen '
 'radio berita operasi terakhir yang mengkritik Ortega, yang pemerintahnya '
 'baru-baru ini menutup tujuh stesen radio Katolik yang berkaitan dengan uskup '
 'yang mengkritik Ortega. "Tidak perlu mematikan peralatan atau dengan '
 'sewenang-wenangnya menarik lesen akan berjaya menampar kita atau '
 'menjerumuskan kebenaran. Jauh dari menakutkan kita, tindakan mereka '
 'menguatkan kita," kata Toruno. Pemerintah Nicaraguan tidak dapat dihubungi '
 'segera untuk memberi komen. Agensi telekomunikasi Nicaraguan, dalam '
 'pernyataan yang disiarkan oleh Toruno, mengatakan stesen radio itu tidak '
 'memen

In [26]:
pprint(quantized_bigbird.greedy_decoder([string]))

['(Reuters) - Kerajaan Nicaraguan memerintahkan penutupan stesen radio yang '
 'terkenal dengan penentangannya terhadap Presiden Daniel Ortega, kata '
 'pengarahnya pada hari Jumaat. Stesen radio tempatan Radio Dario ditutup pada '
 'hari Jumaat, kata pengarahnya Anibal Toruno di akaun Twitternya, menunjukkan '
 'gambar keputusan pemerintah. Radio Dario dianggap sebagai salah satu stesen '
 'radio berita operasi terakhir yang mengkritik Ortega, yang pemerintahnya '
 'baru-baru ini menutup tujuh stesen radio Katolik yang berkaitan dengan uskup '
 'yang mengkritik Ortega. "Tidak ada mematikan peralatan atau dengan '
 'sewenang-wenangnya menarik lesen akan berjaya menampar kita atau '
 'menjerumuskan kebenaran. Jauh dari menakutkan kita, tindakan mereka '
 'menguatkan kita," kata Toruno. Kerajaan Nicaraguan tidak dapat dihubungi '
 'dengan segera untuk memberi komen. Agensi telekomunikasi Nicaraguan, dalam '
 'pernyataan yang disiarkan oleh Toruno, mengatakan stesen radio itu tidak '
 'me

In [27]:
pprint(bigbird_small.greedy_decoder([string2]))

['KUALA LUMPUR (13 Ogos): Kerajaan akan memberi tumpuan untuk meletakkan '
 'projek kapal tempur (LCS) yang dipasang semula di landasan tepat seperti '
 'diputuskan Kabinet, kata Menteri Pertahanan Kanan, Datuk Seri Hishammuddin '
 'Hussein dalam satu hantaran di Facebook. Beliau berkata rundingan dengan '
 'vendor, pengeluar peralatan asal (OEM) dan bank sedang dijalankan. '
 'Hishammuddin berkata demikian berhubung perolehan LCS yang menjadi '
 'kontroversi susulan laporan Jawatankuasa Kira-Kira Wang Negara (PAC) yang '
 'mendedahkan bahawa bukan satu kapal sudah selesai walaupun kontrak projek '
 'itu, bernilai RM6.083 bilion, menetapkan lima daripada enam kapal itu perlu '
 'disiapkan dan diserahkan pada Ogos 2022. Dalam catatan itu, Hishammuddin '
 'berkata pasukannya sudah membawa anggota media ke Boustead Naval Shipyard di '
 'Lumut, Perak pada Sabtu (13 Ogos) bagi melihat keadaan luaran dan dalaman '
 'LCS yang sedang dalam pembinaan. "Saya tidak pernah jemu dengan menegaskan '

In [28]:
pprint(quantized_bigbird_small.greedy_decoder([string2]))

['KUALA LUMPUR (13 Ogos): Kerajaan akan memberi tumpuan untuk meletakkan '
 'projek kapal tempur (LCS) yang dipasang semula di landasan tepat seperti '
 'diputuskan Kabinet, kata Menteri Pertahanan Kanan Datuk Seri Hishammuddin '
 'Hussein dalam satu hantaran di Facebook. Beliau berkata rundingan dengan '
 'vendor, pengeluar peralatan asal (OEM) dan bank sedang dijalankan. '
 'Hishammuddin berkata demikian berhubung perolehan LCS yang menjadi '
 'kontroversi susulan laporan Jawatankuasa Kira-Kira Wang Negara (PAC) yang '
 'mendedahkan bukan satu kapal itu sudah selesai walaupun kontrak projek itu, '
 'bernilai RM6.083 bilion, menetapkan lima daripada enam kapal itu perlu '
 'disiapkan dan diserahkan pada Ogos 2022. Dalam catatan itu, Hishammuddin '
 'berkata pasukannya sudah membawa anggota media ke Boustead Naval Shipyard di '
 'Lumut, Perak pada Sabtu (13 Ogos) bagi melihat keadaan luaran dan dalaman '
 'LCS yang sedang dalam pembinaan. "Saya tidak pernah jemu dengan menegaskan '
 'b

In [29]:
pprint(bigbird.greedy_decoder([string2]))

['KUALA LUMPUR (13 Ogos): Kerajaan akan memberi tumpuan untuk meletakkan '
 'semula projek kapal tempur litatoral (LCS) di landasan seperti yang '
 'diputuskan Kabinet, kata Menteri Pertahanan Kanan, Datuk Seri Hishammuddin '
 'Hussein dalam hantaran di Facebook. Beliau berkata, rundingan dengan vendor, '
 'pengeluar peralatan asal (OEM) dan bank sedang dijalankan. Hishammuddin '
 'berkata demikian berhubung perolehan LCS yang menjadi kontroversi berikutan '
 'laporan Jawatankuasa Kira-Kira Wang Negara (PAC) mendedahkan tidak ada satu '
 'kapal pun yang sudah siap sepenuhnya walaupun kontrak projek itu, bernilai '
 'RM6.083 bilion, menetapkan lima daripada enam kapal itu perlu disiapkan dan '
 'diserahkan Ogos 2022. Dalam hantaran itu, Hishammuddin berkata, pasukannya '
 'sudah membawa anggota media ke Boustead Naval Shipyard di Lumut, Perak pada '
 'Sabtu (13 Ogos) untuk melihat keadaan luar dan dalaman LCS yang sedang dalam '
 'pembinaan. "Saya tidak pernah jemu menegaskan berkali-ka

In [30]:
pprint(quantized_bigbird.greedy_decoder([string2]))

['KUALA LUMPUR (13 Ogos): Kerajaan akan memberi tumpuan untuk meletakkan '
 'semula projek kapal tempur litatoral (LCS) di landasan seperti yang '
 'diputuskan Kabinet, kata Menteri Pertahanan Kanan, Datuk Seri Hishammuddin '
 'Hussein dalam hantaran di Facebook. Beliau berkata rundingan dengan vendor, '
 'pengeluar peralatan asal (OEM) dan bank sedang dijalankan. Hishammuddin '
 'berkata demikian berhubung perolehan LCS yang menjadi kontroversi berikutan '
 'laporan Jawatankuasa Kira-Kira Wang Negara (PAC) mendedahkan tidak ada satu '
 'kapal pun yang sudah siap sepenuhnya walaupun kontrak projek itu, bernilai '
 'RM6.083 bilion, menetapkan lima daripada enam kapal itu perlu disiapkan dan '
 'diserahkan Ogos 2022. Dalam hantaran itu, Hishammuddin berkata pasukannya '
 'sudah membawa anggota media ke Boustead Naval Shipyard di Lumut, Perak pada '
 'Sabtu (13 Ogos) untuk melihat keadaan luaran dan dalaman LCS yang sedang '
 'dalam pembinaan. "Saya tidak pernah jemu menegaskan berkali-ka